# Re-Chunking Larger Datasets 

This notebook extends ideas covered in the [basic workflow](./ReChunkingData.ipynb).  This 
notebook will perfrom the same operations, but will work on the **much** larger dataset, and 
involve some parallelization using the dask scheduler. 

:::{Warning}

You should run this **only** on a cloud compute node -- on ESIP Nebari, for example. We 
will be reading and writing **enormous** amounts of data to S3 buckets. To do that over a 
typical network connection will saturate your bandwidth and take days to complete.

:::

## System Setup 

In [3]:
# Activate logging
import logging
logging.basicConfig(level=logging.INFO, force=True)


## Plumb Data Source
We're going to look at a particular dataset from the National Water Model Reanalysis Version 2.1. 
The dataset is part of the AWS Open Data Program, and is included in the HyTEST data catalog.


In [4]:
import intake
url = 'https://raw.githubusercontent.com/hytest-org/hytest/main/dataset_catalog/hytest_intake_catalog.yml'
cat = intake.open_catalog(url)
cat['nwm21-streamflow-cloud']

INFO:numexpr.utils:NumExpr defaulting to 8 threads.


nwm21-streamflow-cloud:
  args:
    consolidated: true
    storage_options:
      anon: true
    urlpath: s3://noaa-nwm-retrospective-2-1-zarr-pds/chrtout.zarr
  description: National Water Model 2.1 CHRTOUT on AWS
  driver: intake_xarray.xzarr.ZarrSource
  metadata:
    catalog_dir: https://raw.githubusercontent.com/hytest-org/hytest/main/dataset_catalog


## Load the zarr data
We'll take advantage of the `intake` mechanism and load the data 
directly.  We'll need to set up our AWS credentials first, since
this data is stored on an S3 bucket. 

In [5]:
import os
import xarray as xr
import configparser
awsconfig = configparser.ConfigParser()
awsconfig.read(
    os.path.expanduser('~/.aws/credentials') # default location... if yours is elsewhere, change this.
)
_profile_nm  = 'osn-renci'
_endpoint = 'https://renc.osn.xsede.org'
# Set environment vars based on parsed awsconfig
os.environ['AWS_PROFILE'] = _profile_nm
os.environ['AWS_ACCESS_KEY_ID']     = awsconfig[_profile_nm]['aws_access_key_id']    
os.environ['AWS_SECRET_ACCESS_KEY'] = awsconfig[_profile_nm]['aws_secret_access_key']    
## Your profile may require that you specify an endpoint by which  you access S3 object storage
os.environ['AWS_S3_ENDPOINT'] = _endpoint
try: 
    del os.environ['AWS_PROFILE']
except KeyError:
    pass
#ds = cat['nwm21-streamflow-cloud'].to_dask()

In [6]:
import fsspec
# f = fsspec.filesystem("s3", anon=False)
# f.ls("s3://esip-qhub/usgs/hytest")

## Spin up Dask Cluster
Our rechunking operation will be able to work in parallel. To do that, we will
spin up a `dask` cluster on the cloud hardware to schedule the various workers.
Note that this cluster must be configured with a specific user **profile** with 
permissions to write to our eventual output location. 

In [4]:
# %run ../environment_set_up/Start_Dask_Cluster_Nebari.ipynb
import os
import logging

try:
    from dask_gateway import Gateway
except ImportError:
    logging.error("Unable to import Dask Gateway.  Are you running in a cloud compute environment?\n")
    raise
os.environ['DASK_DISTRIBUTED__SCHEDULER__WORKER_SATURATION'] = "1.0"

gateway = Gateway()
_options = gateway.cluster_options()
_options.conda_environment='users/users-pangeo'  ##<< this is the conda environment we use on nebari.
_options.profile = 'Medium Worker'
_env_to_add={}
aws_env_vars=['AWS_ACCESS_KEY_ID',
              'AWS_SECRET_ACCESS_KEY',
              'AWS_SESSION_TOKEN',
              'AWS_DEFAULT_REGION',
              'AWS_S3_ENDPOINT', 
              'DASK_DISTRIBUTED__SCHEDULER__WORKER_SATURATION']
for _e in aws_env_vars:
    if _e in os.environ:
        _env_to_add[_e] = os.environ[_e]
_options.environment_vars = _env_to_add    
cluster = gateway.new_cluster(_options)          ##<< create cluster via the dask gateway
cluster.adapt(minimum=2, maximum=30)             ##<< Sets scaling parameters. 

client = cluster.get_client()

print("The 'cluster' object can be used to adjust cluster behavior.  i.e. 'cluster.adapt(minimum=10)'")
print("The 'client' object can be used to directly interact with the cluster.  i.e. 'client.submit(func)' ")
print(f"The link to view the client dashboard is:\n>  {client.dashboard_link}")

The 'cluster' object can be used to adjust cluster behavior.  i.e. 'cluster.adapt(minimum=10)'
The 'client' object can be used to directly interact with the cluster.  i.e. 'client.submit(func)' 
The link to view the client dashboard is:
>  https://nebari.esipfed.org/gateway/clusters/dev.0338ebba9cbe44518ae21a4633525692/status


## Read Sample Data

In [7]:
import dask
with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    smplData = ds.where(ds.gage_id != ''.rjust(15).encode(), drop=True) # subset to only those features with a valid gage_id
    smplData.drop('crs') # Not needed/wanted for this analysis
smplData

NameError: name 'ds' is not defined

## Re-Chunk Plan
We will configure a new chunking plan which will favor time-series analysis. 
Using the dimensions of the data: 
* 367439 time steps
* 7994 feature IDs

We can write the new plan as: 

In [8]:
# The new chunking plan:
chunk_plan = {
    'streamflow': {'time': 367439, 'feature_id': 1}, # all time records in one chunk for each feature_id
    'velocity': {'time': 367439, 'feature_id': 1},
    'elevation': (7994,),
    'gage_id': (7994,),
    'latitude': (7994,),
    'longitude': (7994,),    
    'order': (7994,),    
    'time': (367439,), # all time coordinates in one chunk
    'feature_id': (7994,) # all feature_id coordinates in one chunk
}


In [9]:
del os.environ['DASK_DISTRIBUTED__SCHEDULER__WORKER_SATURATION']

KeyError: 'DASK_DISTRIBUTED__SCHEDULER__WORKER_SATURATION'

In [22]:
# Manually reset the chunking metadata in prep for re-chunking
#smplData = smplData.chunk(chunks={'feature_id':1, 'time': 367439})
for x in smplData.variables:
    smplData[x].encoding['chunks'] = None

## Set up output location

With this plan, we can ask `rechunker` to re-write the data using the prescribed chunking pattern.

Unlike with the smaller dataset, we need to write this very large dataset to an object store in the datacenter: an S3 'bucket'.  So we need to set that up so that `rechunker` will have a suitable place to write data. This new data will be a complete copy of the original, just re-organized a bit. 

In [10]:
os.environ['AWS_DEFAULT_REGION']='us-west-2'

In [15]:
from getpass import getuser
import fsspec
uname=getuser()

fsw = fsspec.filesystem(
    's3', 
    anon=False, 
    default_fill_cache=False, 
    skip_instance_cache=True, 
    client_kwargs={'endpoint_url': os.environ['AWS_S3_ENDPOINT'], }
)
#workspace= "s3://esip-qhub/usgs/hytest"
os.environ['AWS_PROFILE'] = _profile_nm

workspace = 's3://rsignellbucket2/'
testDir = workspace + "testing/"
myDir = testDir + f'{uname}/'
fsw.mkdir(testDir)
fsw.ls(testDir)

INFO:aiobotocore.credentials:Found credentials in environment variables.


['rsignellbucket2/testing/02_kerchunk.ipynb',
 'rsignellbucket2/testing/ATL08_20181014084920_02400109_003_01.h5',
 'rsignellbucket2/testing/cluster_conf.py',
 'rsignellbucket2/testing/cog',
 'rsignellbucket2/testing/combine_files_tpBiasCorr.csh',
 'rsignellbucket2/testing/foo.json',
 'rsignellbucket2/testing/gzt5142',
 'rsignellbucket2/testing/ortho_2021_10_7_sageLot_noalpha_clip_cog.tif',
 'rsignellbucket2/testing/time']

In [16]:
for fn in ['rechunked.zarr', 'staging.zarr', 'writetest.csv']:
    if fsw.exists(myDir + fn):
        fsw.rm(myDir + fn, recursive=True)


In [19]:
import zarr
#staging = fsw.get_mapper(myDir + 'staging.zarr')
#del os.environ['AWS_PROFILE']
os.environ['AWS_PROFILE'] = _profile_nm

staging = zarr.storage.FSStore(myDir + 'staging.zarr', mode="w")

outfile = fsw.get_mapper(myDir + 'rechunked.zarr')

In [20]:
zarr.group(staging)

PermissionError: Forbidden

## Ready to rechunk

In [65]:
import rechunker
## Recall that merely invoking rechunker does not do any work... just sorts out 
## the rechunking plan and writes metadata.
result = rechunker.rechunk(
    smplData,
    chunk_plan,
    "2GB",
    outfile, 
    temp_store=staging, 
)

ReadOnlyError: object is read-only

In [66]:
f=myDir + "write_test.txt"
fsw.touch(f)

{'ResponseMetadata': {'RequestId': 'tx0000038f5da0f45a7b4b4-00640b5584-a9e24-default',
  'HostId': '',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-length': '0',
   'etag': '"d41d8cd98f00b204e9800998ecf8427e"',
   'accept-ranges': 'bytes',
   'x-amz-request-id': 'tx0000038f5da0f45a7b4b4-00640b5584-a9e24-default',
   'date': 'Fri, 10 Mar 2023 16:06:29 GMT'},
  'RetryAttempts': 0},
 'ETag': '"d41d8cd98f00b204e9800998ecf8427e"'}

In [67]:
fsw.ls(myDir)

['rsignellbucket2/testing/gzt5142/write_test.txt']

In [9]:
import zarr
zarr.group(store=staging, overwrite=True, )

TypeError: group() got an unexpected keyword argument 'mode'

In [10]:
staging

In [50]:
type(outfile)


fsspec.mapping.FSMap

In [ ]:
from dask.distributed import progress, performance_report

with performance_report(filename="dask-report.html"):
    r = result.execute(retries=10)  

In [ ]:
import zarr
_ = zarr.consolidate_metadata(outfile)

## Results
Let's read in the resulting re-chunked dataset to see how it looks:

In [34]:
reChunkedData = xr.open_zarr(outfile)
reChunkedData

FileNotFoundError: No such file or directory: '<fsspec.mapping.FSMap object at 0x7f151aa90430>'

### Comparison


In [ ]:
## Before:
sampleData['streamflow'].sel(feature_id=1343034)
# Note: three chunks needed to service a single feature_id


In [ ]:
## After:
reChunkedData['streamflow'].sel(feature_id=1343034) 
# All data for the specified feature_id is in a single chunk


In [21]:
client.close()
cluster.close()

In [14]:

def myAWS_Credentials():
    """Test function to return AWS credential information."""
    return {
    "AWS_PROFILE": os.environ.get("AWS_PROFILE", "<not set>"),
    "AWS_ACCESS_KEY_ID": os.environ.get('AWS_ACCESS_KEY_ID', '<not set>'),
    "AWS_S3_ENDPOINT": os.environ.get('AWS_S3_ENDPOINT', '<not set>')    
}


In [15]:
myAWS_Credentials()

{'AWS_PROFILE': 'osn-renci',
 'AWS_ACCESS_KEY_ID': '8A852VG4EG6NHHM4WUDX',
 'AWS_S3_ENDPOINT': 'https://renc.osn.xsede.org'}

In [16]:
client.submit(myAWS_Credentials).result()

NameError: name 'client' is not defined

In [72]:
client.submit(fsw.touch, myDir+"write_test_2.txt").result()

{'ResponseMetadata': {'RequestId': 'tx00000e1d4f381c02d61b6-00640b55b5-132e6e-default',
  'HostId': '',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-length': '0',
   'etag': '"d41d8cd98f00b204e9800998ecf8427e"',
   'accept-ranges': 'bytes',
   'x-amz-request-id': 'tx00000e1d4f381c02d61b6-00640b55b5-132e6e-default',
   'date': 'Fri, 10 Mar 2023 16:07:17 GMT'},
  'RetryAttempts': 2},
 'ETag': '"d41d8cd98f00b204e9800998ecf8427e"'}

In [73]:
fsw.ls(myDir)

['rsignellbucket2/testing/gzt5142/write_test.txt']

In [1]:
import zarr
zarr.__version__

'2.13.6'